In [1]:
import numpy as np
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
# testの答えはゼロで詰めとく
test['Survived'] = np.nan

data = train.append(test, ignore_index=True, sort=False)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
# data.describe()
data.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [7]:
data['Fare'].fillna(8, inplace= True)
data['Ticket_count'] = data.Ticket.apply(lambda x: data[data['Ticket']==x].shape[0])
data['Fare_ticket'] = data.apply(lambda x: x.Fare/x.Ticket_count,axis=1)
data['Fare_bin'] = pd.qcut(data['Fare_ticket'], 4, labels=('Fare_bin1', 'Fare_bin2', 'Fare_bin3', 'Fare_bin4'))

In [9]:
# 新しいタイトル特徴量を作る
data['Title'] = data['Name'].str.split(', ').str[1].str.split('.').str[0]
# 女性をひとくくりに
data['Title'] = data['Title'].replace(['Ms', 'Mlle'], 'Miss')
data['Title'] = data['Title'].replace(['Mme'], 'Mrs')
data['Title'] = data['Title'].replace(['Dona', 'Dr', 'Rev','the Countess','Capt','Lady','Sir','Jonkheer','Don','Major','Col'], 'Rare')

# 別のやり方
# 敬称(honorific)を抽出し、敬称ごとの数を調べる
# train['honorific'] = train['Name'].map(lambda x: x.split(', ')[1].split('. ')[0])
# train['honorific'].value_counts() 

In [13]:
def Cal_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):
        
        if Pclass == 1:
            return 37
        elif Pclass == 2:
            return 29
        
        else:
            return 24
        
    else:
        return Age
    
print(data['Title'].head(10))
data['Age']= data[['Age', 'Title']].apply(Cal_age, axis=1)
data['Age_bin'] = pd.cut(data['Age'].astype(int), 5, labels=('Age_bin1', 'Age_bin2','Age_bin3','Age_bin4','Age_bin5'))

0        Mr
1       Mrs
2      Miss
3       Mrs
4        Mr
5        Mr
6        Mr
7    Master
8       Mrs
9       Mrs
Name: Title, dtype: object


In [14]:
print(data['Age'].head(10))

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5    24.0
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64


In [15]:
print(data['Age_bin'].head(10))

0    Age_bin2
1    Age_bin3
2    Age_bin2
3    Age_bin3
4    Age_bin3
5    Age_bin2
6    Age_bin4
7    Age_bin1
8    Age_bin2
9    Age_bin1
Name: Age_bin, dtype: category
Categories (5, object): [Age_bin1 < Age_bin2 < Age_bin3 < Age_bin4 < Age_bin5]


In [16]:
# 家族の大きさの特徴量
def Cal_Family_bin(cols):
    FamilySize = cols[0] +cols[1]
    if FamilySize == 0:
        return 'Alone'
    elif 1 <= FamilySize <= 3:
        return 'Family'
    elif FamilySize >= 4:
        return 'Big_family'
    
data['Family_type'] = data[['SibSp', 'Parch']].apply(Cal_Family_bin, axis=1)

data.loc[data['Embarked'].isnull(), 'Embarked'] = 'S'

In [17]:
data['Family_name'] = data['Name'].str.split(', ').str[0] + '_' + data['Fare'].astype(str)

list1 = data[((data['Sex'] == 'female') | (data['Age'] < 14)) & (data['Survived']==0)]['Family_name'].tolist()

list2 = data[(data['Sex'] == 'male') & (data['Age'] > 18) & (data['Survived'] == 1)]['Family_name'].tolist()

In [18]:
def FC_dead(row):
    if row['Family_name'] in list1:
        return 1
    else:
        return 0
    
def M_Alive(row):
    if row['Family_name'] in list2:
        return 1
    else:
        return 0

data['Family_wit_FC_dead'] = data.apply(FC_dead, axis = 1)
data['Family_wit_M_alive'] = data.apply(M_Alive, axis = 1)

In [19]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Embarked,Ticket_count,Fare_ticket,Fare_bin,Title,Age_bin,Family_type,Family_name,Family_wit_FC_dead,Family_wit_M_alive
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,S,1,7.25000,Fare_bin1,Mr,Age_bin2,Family,Braund_7.25,0,0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,C,2,35.64165,Fare_bin4,Mrs,Age_bin3,Family,Cumings_71.2833,0,0
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,S,1,7.92500,Fare_bin2,Miss,Age_bin2,Alone,Heikkinen_7.925,0,0
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,S,2,26.55000,Fare_bin4,Mrs,Age_bin3,Family,Futrelle_53.1,0,0
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,S,1,8.05000,Fare_bin2,Mr,Age_bin3,Alone,Allen_8.05,0,0


In [20]:
Fare_bin = pd.get_dummies(data['Fare_bin'])
Pclass_bin = pd.get_dummies(data['Pclass'], prefix='Class')
Title_bin = pd.get_dummies(data['Title'])
Sex_bin = pd.get_dummies(data['Sex'], drop_first= True, prefix = 'Sex')
Age_bin = pd.get_dummies(data['Age_bin'])
Family_type = pd.get_dummies(data['Family_type'])
Embarked_bin = pd.get_dummies(data['Embarked'], prefix='Embarked')
Family_wit_FC_dead=data['Family_wit_FC_dead'].astype(np.uint8)
Family_wit_M_alive=data['Family_wit_M_alive'].astype(np.uint8)

In [21]:
data_cleaned = pd.concat([data['Survived'], Fare_bin, Pclass_bin, Title_bin, Sex_bin, Age_bin,Family_type,Embarked_bin,Family_wit_FC_dead,Family_wit_M_alive],axis=1)

train_cleaned = data_cleaned[data['Survived'].notnull()]
test_cleaned = data_cleaned[data['Survived'].isnull()]

test_cleaned.drop('Survived', axis=1, inplace=True)
PassId = test['PassengerId']

C:\Users\fujii\Anaconda3\envs\tesorflow-gpu\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [22]:
data_cleaned.head()

,Survived,Fare_bin1,Fare_bin2,Fare_bin3,Fare_bin4,Class_1,Class_2,Class_3,Master,Miss,...,Age_bin4,Age_bin5,Alone,Big_family,Family,Embarked_C,Embarked_Q,Embarked_S,Family_wit_FC_dead,Family_wit_M_alive
0,0.0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
1,1.0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
2,1.0,0,1,0,0,0,0,1,0,1,...,0,0,1,0,0,0,0,1,0,0
3,1.0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,0.0,0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix,mean_absolute_error

X = train_cleaned.drop('Survived', axis=1)
y = train_cleaned['Survived']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

0.9777777777777777

In [25]:
from xgboost import XGBClassifier
classifier= XGBClassifier(colsample_bylevel=0.9999999,
                         colsample_bytree=0.9999999,
                         gamma=0.99999,
                         max_depth=5,
                         min_child_weight=1,
                         n_estimators=1000,
                         nthread=4,
                         random_state=2,
                         silent=True)
classifier.fit(X,y)
classifier.score(X_test, y_test)

0.9777777777777777

In [27]:
xgb_predict = classifier.predict(test_cleaned)
sample = pd.DataFrame(xgb_predict)

In [28]:
sample.head(7)

,0
0,0.0
1,1.0
2,0.0
3,0.0
4,1.0
5,0.0
6,1.0
